In [9]:
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool
import math, re

Connect LangChain to Ollama

In [3]:

# Connect to local Llama 3 model
llm = Ollama(model="mistral")
response = llm.invoke("Explain what is Agentic AI in one line.")
print(response)

/tmp/ipykernel_33019/252850057.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


 Agentic AI refers to artificial intelligence systems that have the ability to perceive their environment, make decisions, and take actions with the goal of achieving specific objectives autonomously.


Add Agent Capabilities (Tools)

In [10]:
# Define a simple, safe calculator tool
@tool
def simple_calculator(expression: str) -> str:
    """Safely evaluate a basic math expression (supports sqrt, %, +, -, *, /)."""
    try:
        expr = expression.lower()
        expr = re.sub(r"(\d+)\s*%", r"(\1/100)", expr)  # Convert 20% → (20/100)
        expr = expr.replace("sqrt", "math.sqrt")

        result = eval(expr, {"__builtins__": None, "math": math})
        return str(result)
    except Exception as e:
        return f"Error: {e}"

# Add the tool to the agent
tools = [simple_calculator]

In [11]:
tools

[StructuredTool(name='simple_calculator', description='Safely evaluate a basic math expression (supports sqrt, %, +, -, *, /).', args_schema=<class 'langchain_core.utils.pydantic.simple_calculator'>, func=<function simple_calculator at 0x7538b38a1760>)]

In [22]:
# Initialize an agent
agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)

In [23]:
agent

AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nsimple_calculator(expression: str) -> str - Safely evaluate a basic math expression (supports sqrt, %, +, -, *, /).\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [simple_calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'), llm=Ollama(model='mistral'), output_parser=StrOutp

In [24]:
# Try it out
try:
    result = agent.run("What is the square root of 144, plus 20% of 50?")
    print("✅ Result:", result)
except Exception as e:
    print("Handled gracefully:", e)



> Entering new AgentExecutor chain...
 To find the square root of 144 and then add 20% of 50, I need to perform two calculations. First, I'll find the square root using the `simple_calculator` tool. Then, I'll calculate 20% of 50.

Action: simple_calculator
Action Input: sqrt(144)
Observation: 12.0
Thought: Now that I know the square root of 144, I can proceed to find 20% of 50.

Action: simple_calculator
Action Input: 50*20/100
Observation: 10.0
Thought: I have now completed both calculations. Now, I will add the square root of 144 and 20% of 50 to get the final answer.

Final Answer: The square root of 144 plus 20% of 50 is 22.0.

> Finished chain.
✅ Result: The square root of 144 plus 20% of 50 is 22.0.
